# **ADVANCED MACHINE LEARNING FINAL PROJECT**

##### Authors: Dario Della Mura - David Doci

##### Course: Advanced Machine Learning

##### Master's Degree: Data Science

##### Anno accademico: 2020/2021

Setting Colab GPU environment

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Feb  9 19:45:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Import Libreries and Fruit-360 Dataset

In [ ]:
# import Fruits-360 dataset from Github

!git clone 'https://github.com/Horea94/Fruit-Images-Dataset.git'

fatal: destination path 'Fruit-Images-Dataset' already exists and is not an empty directory.


In [ ]:
# generiche 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import tensorflow as tf
import random
import glob
import cv2

# keras
import keras
from keras.models import Sequential
from keras import applications
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Dropout, Activation, AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
import keras.backend as K
from tensorflow.keras.utils import to_categorical
from keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from keras import Model

# Sklearn
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.estimator_checks import check_estimator

import pickle

np.random.seed(1234)
tf.random.set_seed(1234)

## Data pre-processing

In [ ]:
# Pre-processing on training images and building images vector and classes vector
%%time

path_train = '/content/Fruit-Images-Dataset/Training/*'

fruit_train = []
fruit_label = []
for dir_path in glob.glob(path_train):
    img_label = dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (50,50))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        fruit_train.append(image)
        fruit_label.append(img_label)
fruit_train = np.array(fruit_train)
fruit_label = np.array(fruit_label)

CPU times: user 15.7 s, sys: 4.3 s, total: 20 s
Wall time: 37.9 s


In [ ]:
# class codification
%%time

# numerical label assignment to nominal classes
label_to_id = {v:k for k,v in enumerate(np.unique(fruit_label))}
id_to_label = {v:k for k,v in label_to_id.items() }
# assignment of numerical labels to the reference images
fruit_label_id = np.array([label_to_id[i] for i in fruit_label])

CPU times: user 75.6 ms, sys: 897 ms, total: 972 ms
Wall time: 1.45 s


## Load VGG16 Model

In [ ]:
# import vgg16 model
base_model = VGG16(input_shape=(50,50,3),include_top = False, weights='imagenet')

In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 50, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 50, 50, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 50, 50, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 25, 25, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 25, 25, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 25, 25, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

## Features Extraction

In [ ]:
# cut vgg16 architecture on "block4_pool" layer
%%time
model_block4_pool = Model(inputs = base_model.input, outputs = base_model.get_layer('block4_pool').output)

CPU times: user 5.74 ms, sys: 0 ns, total: 5.74 ms
Wall time: 13.5 ms


In [ ]:
%%time
model_block4_pool.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 50, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 50, 50, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 50, 50, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 25, 25, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 25, 25, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 25, 25, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [ ]:
# features extraction on train set 
%%time
block4_pool_features = model_block4_pool.predict(fruit_train)

CPU times: user 10.3 s, sys: 2.78 s, total: 13 s
Wall time: 24.2 s


In [ ]:
# reshape of features extracted
%%time
block4_pool_features = block4_pool_features.reshape((block4_pool_features.shape[0], (block4_pool_features.shape[1]*block4_pool_features.shape[2]*block4_pool_features.shape[3])))

CPU times: user 40 µs, sys: 11 µs, total: 51 µs
Wall time: 55.1 µs


In [ ]:
# splitting in train(X_train, y_train) and val(X_test, y_test)
%%time
X_train, X_test, y_train, y_test = train_test_split(block4_pool_features, fruit_label_id, test_size=0.1, random_state=101)

CPU times: user 354 ms, sys: 66.5 ms, total: 420 ms
Wall time: 423 ms


## Train Traditional ML Models: SVM - Logistic Regression - KNN

In [ ]:
%%time

#Classifiers definition
classificatore_svm = sklearn.svm.SVC()
classificatore_lr = LogisticRegression()
classificatore_knn = KNeighborsClassifier()

CPU times: user 48 µs, sys: 12 µs, total: 60 µs
Wall time: 63.7 µs


In [ ]:
%%time

# training SVM with features extracted
clf_svm = classificatore_svm.fit(X_train, y_train)

CPU times: user 15min 56s, sys: 3.99 s, total: 16min
Wall time: 15min 56s


In [ ]:
%%time

# training Logtic regression with features extracted
clf_lr = classificatore_lr.fit(X_train, y_train)

CPU times: user 18min 5s, sys: 51.9 s, total: 18min 57s
Wall time: 9min 50s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
%%time

# training KNN with features extracted
clf_knn = classificatore_knn.fit(X_train, y_train)

CPU times: user 234 ms, sys: 14 µs, total: 234 ms
Wall time: 234 ms


## Testing Models

In [ ]:
%%time

# Pred with SVM the validation set(X_test)
y_pred_svm = clf_svm.predict(X_test)

CPU times: user 19min 4s, sys: 1.12 s, total: 19min 5s
Wall time: 19min 17s


In [ ]:
%%time

# Pred with LR the validation set(X_test)
y_pred_lr = clf_lr.predict(X_test)

CPU times: user 558 ms, sys: 29 ms, total: 587 ms
Wall time: 332 ms


In [ ]:
%%time

# Pred KNN the validation set(X_test)
y_pred_knn = clf_knn.predict(X_test)

CPU times: user 3min 17s, sys: 7.34 s, total: 3min 24s
Wall time: 1min 47s


In [ ]:
import pickle

# save svm model to disk
svm = 'svm.sav'
pickle.dump(clf_svm, open(svm, 'wb'))

# save lr model to disk
lr = 'lrl.sav'
pickle.dump(clf_lr, open(lr, 'wb'))

# save knn model to disk
knn = 'knn.sav'
pickle.dump(clf_knn, open(knn, 'wb'))

In [ ]:
import shutil 
#shutil.move("/content/knn.sav", "/content/drive/MyDrive/Colab Notebooks")
#shutil.move("/content/svm.sav", "/content/drive/MyDrive/Colab Notebooks")
#shutil.move("/content/lrl.sav", "/content/drive/MyDrive/Colab Notebooks")

'/content/drive/MyDrive/Colab Notebooks/lrl.sav'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Results on Val

In [ ]:
# Scores of traditional ML models
score_svm = accuracy_score(y_test, y_pred_svm)
score_lr = accuracy_score(y_test, y_pred_lr)
score_knn = accuracy_score(y_test, y_pred_knn)

In [ ]:
dict_score = {"Score_svm": score_svm, "Score_lr": score_lr, "Score_knn": score_knn}

for k, v in dict_score.items():
    print(str(k) + " = " + str(v))

Score_svm = 0.9998522895125553
Score_lr = 0.9997045790251108
Score_knn = 0.9998522895125553


## Testing On Unseen Images

In [ ]:
# Pre-processing on test images and building images vector and classes vector
%%time

path_test = '/content/Fruit-Images-Dataset/Test/*'

fruit_test = []
fruit_test_label = []
for dir_path in glob.glob(path_test):
    img_label = dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(dir_path,"*.jpg")):
        image = cv2.imread(image_path,cv2.IMREAD_COLOR)
        image = cv2.resize(image, (50,50))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        fruit_test.append(image)
        fruit_test_label.append(img_label)
fruit_test = np.array(fruit_test)
fruit_test_label = np.array(fruit_test_label)

CPU times: user 5.04 s, sys: 820 ms, total: 5.86 s
Wall time: 7.17 s


In [ ]:
# class codification
%%time

# numerical label assignment to nominal classes
label_to_id_test = {v:k for k,v in enumerate(np.unique(fruit_test_label)) }
id_to_label_test = {v:k for k,v in label_to_id_test.items() }
# assignment of numerical labels to the reference images
fruit_label_test_id = np.array([label_to_id_test[i] for i in fruit_test_label])

CPU times: user 14.7 ms, sys: 8.35 ms, total: 23 ms
Wall time: 23.7 ms


In [ ]:
# features extraction on train set 
%%time
block4_pool_features_test = model_block4_pool.predict(fruit_test)

CPU times: user 3.1 s, sys: 661 ms, total: 3.76 s
Wall time: 10.6 s


In [ ]:
# reshape of features extracted
%%time
block4_pool_features_test = block4_pool_features_test.reshape((block4_pool_features_test.shape[0], (block4_pool_features_test.shape[1]*block4_pool_features_test.shape[2]*block4_pool_features_test.shape[3])))

CPU times: user 24 µs, sys: 6 µs, total: 30 µs
Wall time: 35.3 µs


In [ ]:
block4_pool_features.shape, block4_pool_features_test.shape

((67692, 4608), (22688, 4608))

In [ ]:
clf_svm = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/svm.sav', 'rb'))

In [ ]:
# Pred with SVM the test set
%%time
y_pred_test_svm = clf_svm.predict(block4_pool_features_test)

CPU times: user 1h 5min 56s, sys: 3.14 s, total: 1h 5min 59s
Wall time: 1h 5min 46s


In [ ]:
from sklearn.metrics import confusion_matrix
# Confusion matrix
print('Confusion matrix svm:')
cm_svm = confusion_matrix(fruit_label_test_id, y_pred_test_svm)
df_cm_svm = pd.DataFrame(cm_svm)
df_cm_svm.to_excel('cf_svm.xlsx')

Confusion matrix svm:


In [ ]:
clf_lr = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/lrl.sav', 'rb'))

In [ ]:
# Pred with LR the test set
%%time
y_pred_test_lr = clf_lr.predict(block4_pool_features_test)

CPU times: user 1.94 s, sys: 138 ms, total: 2.07 s
Wall time: 1.24 s


In [ ]:
# Confusion matrix
print('Confusion matrix lr:')
cm_lr = confusion_matrix(fruit_label_test_id, y_pred_test_lr)
df_cm_lr = pd.DataFrame(cm_lr)
df_cm_lr.to_excel('cf_lr.xlsx')

Confusion matrix lr:


In [ ]:
clf_knn= pickle.load(open('/content/drive/MyDrive/Colab Notebooks/knn.sav', 'rb'))

In [ ]:
# Pred with KNN the test set
%%time
y_pred_test_knn = clf_knn.predict(block4_pool_features_test)

CPU times: user 10min 45s, sys: 23.4 s, total: 11min 9s
Wall time: 6min 7s


In [ ]:
# Confusion matrix
print('Confusion matrix knn:')
cm_knn = confusion_matrix(fruit_label_test_id, y_pred_test_knn)
df_cm_knn = pd.DataFrame(cm_knn)
df_cm_knn.to_excel('cf_kmm.xlsx')

Confusion matrix knn:


### Results on test

In [ ]:
# Scores of traditional ML Models on test set
score_svm_test = accuracy_score(fruit_label_test_id, y_pred_test_svm)
score_lr_test = accuracy_score(fruit_label_test_id, y_pred_test_lr)
score_knn_test = accuracy_score(fruit_label_test_id, y_pred_test_knn)

In [ ]:
dict_score_test = {"Score_svm_test": score_svm_test, "Score_lr_test": score_lr_test, "Score_knn_test": score_knn_test}

for k, v in dict_score_test.items():
    print(str(k) + " = " + str(v))

Score_svm_test = 0.9776974612129761
Score_lr_test = 0.9592295486600846
Score_knn_test = 0.9518688293370945
